<a href="https://colab.research.google.com/github/Harphies/AI/blob/main/notebooks/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check hardware accelerator/GPU specs
!nvidia-smi

Tue Jan 28 15:27:56 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import time
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import models

# select GPU if Cuda is available
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

Project: A pretrained model of VGG16 trained on ImageNet dataset to perform transfer learning on CIFAR10 dataset
Select the layers to retrain
Replace the outlayer with a layer that fit your contect
And Retrain the model with your data

In [ ]:
# device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# hyparameter
RANDOM_SEED = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
BATCH_SIZE = 128

# model architecture parameter
NUM_CLASSES = 10

# dataser -> CIFAR10
custom_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='data',
                                 train=True,
                                 transform=custom_transform,
                                 download=True)

test_dataset = datasets.CIFAR10(root='data',
                               train=False,
                               transform=custom_transform)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                        batch_size=BATCH_SIZE,
                        num_workers=8,
                        shuffle=False)

# checking the dataset
for images, labels in train_loader:
    print('Image Batch Dimension', images.shape)
    print('Image label Dimension:', labels.shape)
    break

Files already downloaded and verified
Image Batch Dimension torch.Size([128, 3, 224, 224])
Image label Dimension: torch.Size([128])


In [ ]:
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
for param in model.parameters():
    param .requires_grad = False

In [ ]:
# Training the penultimate layer
model.classifier[3].requires_grad = True

In [ ]:
# Replace the output layer
model.classifier[6] = nn.Sequential(
                     nn.Linear(4096, 512),
                     nn.ReLU(),
                     nn.Dropout(0.5),
                     nn.Linear(512, NUM_CLASSES))

In [ ]:
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_predictions, num_examples = 0, 0
    for i, (features, labels) in enumerate(data_loader):
        features = features.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(features)
        _, predicted_labels = torch.max(outputs, 1)
        num_examples +=labels.size(0)
        correct_predictions += (predicted_labels == labels).sum()
    return correct_predictions.float()/num_examples *100

In [ ]:
def compute_epoch_loss(model, data_loader):
    model.eval()
    curr_loss, num_exaples = 0., 0
    with torch.no_grad():
        for features, labels in data_loader:
            features = features.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(features)
            loss = F.cross_entropy(outputs, labels, reduction='sum')
            num_examples +=labels.size(0)
            curr_loss +=loss
        curr_loss = curr_loss / num_examples
        return curr_loss

In [ ]:
start_time = time.time()
total_step = len(train_loader)

for epoch in range(NUM_EPOCHS):
    for i, (features, labels) in enumerate(train_loader):
        features = features.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward and back pass
        outputs = model(features)
        loss = F.cross_entropy(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging
        if not i % 50:
            print('Epoch: %03d/%03d | Batch %04d/%04d | Loss: %.4f'
                 %(epoch+1, NUM_EPOCHS, i, total_step, loss))
    model.eval()
    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d | Train: %.3f%% | Loss: %.3f' %(
        epoch+1, NUM_EPOCHS, compute_accuracy(model, train_loader),
        compute_epoch_loss(model, train_loader)))

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
print('Total Training Time: %.2f min'% ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 0000/0391 | Loss: 0.5176
Epoch: 001/010 | Batch 0050/0391 | Loss: 0.5248
Epoch: 001/010 | Batch 0100/0391 | Loss: 0.5281
Epoch: 001/010 | Batch 0150/0391 | Loss: 0.5064
Epoch: 001/010 | Batch 0200/0391 | Loss: 0.3871
Epoch: 001/010 | Batch 0250/0391 | Loss: 0.5217
Epoch: 001/010 | Batch 0300/0391 | Loss: 0.4867
Epoch: 001/010 | Batch 0350/0391 | Loss: 0.4095


NameError: ignored